In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import mlflow


In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('my-test-experiment')

2025/12/08 06:46:39 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/08 06:46:39 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp-project/02-experiment-tracking/mlruns/1', creation_time=1764872639167, experiment_id='1', last_update_time=1764872639167, lifecycle_stage='active', name='my-test-experiment', tags={}>

In [3]:
def read_dataframe(url):
    df = pd.read_parquet(url)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'

Y_train = df_train[target].values
Y_val = df_val[target].values

In [9]:
with mlflow.start_run(run_name = 'LinearReg'):
    mlflow.set_tag('developer', 'Peter Peto')
    mlflow.log_param('train_data_path', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
    mlflow.log_param('val_data_path', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

    lr = LinearRegression()
    lr.fit(X_train, Y_train)
    
    Y_pred = lr.predict(X_val)
    
    rmse_lin_reg = root_mean_squared_error(Y_val, Y_pred)

    mlflow.log_metric('rmse', rmse_lin_reg)

In [10]:
rmse_lin_reg

7.758715208009878

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [12]:
# Model store as artifact in MLFlow
with mlflow.start_run(run_name = 'LinearReg'):
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [13]:
# with mlflow.start_run(run_name = 'Lasso'):
#     mlflow.set_tag('developer', 'Peter Peto')
#     mlflow.log_param('train_data_path', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
#     mlflow.log_param('val_data_path', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
    
#     lr = Lasso(0.01)
#     lr.fit(X_train, Y_train)
    
#     Y_pred_lass = lr.predict(X_val)
    
#     rmse_lasso = root_mean_squared_error(Y_val, Y_pred_lass)
    
#     mlflow.log_metric('rmse', rmse)

In [14]:
# rmse_lasso

In [15]:
# import xgboost as xgb

In [16]:
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from hyperopt.pyll import scope

In [17]:
# train = xgb.DMatrix(X_train, label=Y_train)
# valid = xgb.DMatrix(X_val, label=Y_val)

In [18]:
# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag("model", "xgboost")
#         mlflow.log_params(params)
#         booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=1000,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=50
#         )
#         Y_pred_booster = booster.predict(valid)
#         rmse_booester = root_mean_squared_error(Y_val, Y_pred_booster)
#         mlflow.log_metric("rmse", rmse_booester)

#     return {'loss': rmse_booester, 'status': STATUS_OK}

In [19]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed': 42
# }

# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials()
# )